### Install Runhouse

In [ ]:
!pip install runhouse

In [ ]:
import runhouse as rh

### Login to Runhouse to load in secrets.

In [ ]:
# You can add token=<your token> if you want to be able to run this without pasting into stdin
rh.login(download_secrets=True, download_config=True, interactive=True)

In [ ]:
# Only if you're using GCP and running inside Colab!
!gcloud init
!gcloud auth application-default login
!cp -r /content/.config/* ~/.config/gcloud

In [ ]:
# Check that secrets are loaded in properly and at least one cloud is ready to use.
!sky check

# Dreambooth Training

Start by creating our training service.

In [12]:
gpu = rh.cluster(name='rh-a10x', instance_type='A100:1')  # On GCP and Azure
# gpu = rh.cluster(name='rh-a10x', instance_type='g5.2xlarge', provider='aws')  # On AWS

training_function_gpu = rh.send(
    fn='https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/train_dreambooth.py:main',
    hardware=gpu,
    reqs=['pip:./diffusers',
          'torch --upgrade --extra-index-url https://download.pytorch.org/whl/cu116',
          'torchvision --upgrade --extra-index-url https://download.pytorch.org/whl/cu116', 
          'transformers', 'accelerate', 'datasets'],
    name='train_dreambooth')
gpu.run_python(['import torch; torch.backends.cuda.matmul.allow_tf32 = True; '
                'torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True'])

Output()

INFO | 2023-01-19 03:45:48,201 | Setting up Send on cluster.
INFO | 2023-01-19 03:45:59,213 | Installing packages on cluster rh-a10x: ['GitPackage: https://github.com/huggingface/diffusers.git@main', 'pip:./diffusers', 'torch --upgrade --extra-index-url https://download.pytorch.org/whl/cu116', 'torchvision --upgrade --extra-index-url https://download.pytorch.org/whl/cu116', 'transformers', 'accelerate', 'datasets']
INFO | 2023-01-19 03:46:04,426 | Send setup complete.
INFO | 2023-01-19 03:46:04,442 | Running command on rh-a10x: python3 -c "import torch; torch.backends.cuda.matmul.allow_tf32 = True; torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True"


Next, we need to upload some images. You want 10-20 images, as close to 512x512 as possible.

In [ ]:
from google.colab import files
import shutil
from pathlib import Path

uploaded = files.upload()

Now we'll send those images to our cluster.

In [4]:
input_images_dir = 'instance_images'
images_path = Path(input_images_dir)
images_path.mkdir(exist_ok=True)

for filename in uploaded.keys():
  shutil.move(filename, images_path / filename)

In [10]:
remote_image_dir = 'dreambooth/instance_images'
rh.folder(url=input_images_dir).to(fs=gpu, url=remote_image_dir)

INFO | 2023-01-19 03:43:54,407 | Copying folder from file:///content/instance_images to rh-a10x://dreambooth/instance_images


Now we'll generate the arguments into the training function (and call this function on the cluster to avoid having to clone it down locally).

In [17]:
class_name = 'person'
create_train_args = rh.send(
    fn='https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/train_dreambooth.py:parse_args',
    hardware=gpu, reqs=[])
train_args = create_train_args(input_args=['--pretrained_model_name_or_path', 'stabilityai/stable-diffusion-2-base',
                                            '--instance_data_dir', remote_image_dir,
                                            '--instance_prompt', f'a photo of sks {class_name}'])
train_args.train_text_encoder = True
train_args.class_data_dir = 'dreambooth/class_images'
train_args.output_dir = 'dreambooth/output'
train_args.mixed_precision = 'bf16'
train_args.with_prior_preservation = True
train_args.prior_loss_weight = 1.0
train_args.class_prompt = f"a photo of {class_name}"
train_args.resolution = 512
train_args.train_batch_size = 4
train_args.gradient_checkpointing = True
train_args.learning_rate = 1e-6
train_args.lr_scheduler = "constant"
train_args.lr_warmup_steps = 0
train_args.num_class_images = 200
train_args.checkpointing_steps = 400
# train_args.resume_from_checkpoint = 'latest'
train_args.max_train_steps = 800

INFO | 2023-01-19 03:57:20,531 | Setting up Send on cluster.
INFO | 2023-01-19 03:57:20,534 | Installing packages on cluster rh-a10x: ['GitPackage: https://github.com/huggingface/diffusers.git@main']
INFO | 2023-01-19 03:57:20,589 | Send setup complete.
INFO | 2023-01-19 03:57:20,595 | Running anonymous send via SSH
INFO | 2023-01-19 03:57:20,695 | Time to send message: 0.1 seconds


And initiate training. This takes around 20 minutes to run.

In [22]:
training_function_gpu(train_args)

INFO | 2023-01-19 04:04:49,255 | Running train_dreambooth via SSH
INFO | 2023-01-19 04:20:54,106 | Time to send message: 964.85 seconds


# Inference

Now we can use our existing Stable Diffusion service to run inferences on this model:

In [23]:
generate_gpu = rh.send(name='sd_generate')

W 01-19 04:21:22 backend_utils.py:434] ~/.ssh/config contains host named rh-a10x.
WARNING | 2023-01-19 04:21:22,125 | ~/.ssh/config contains host named rh-a10x.
W 01-19 04:21:22 backend_utils.py:437] Using 34.70.45.62 to identify host instead.
WARNING | 2023-01-19 04:21:22,132 | Using 34.70.45.62 to identify host instead.


Output()

INFO | 2023-01-19 04:21:37,323 | Setting up Send on cluster.
INFO | 2023-01-19 04:21:48,329 | Copying local package content to cluster <rh-a10x>
INFO | 2023-01-19 04:21:50,312 | Installing packages on cluster rh-a10x: ['./']
INFO | 2023-01-19 04:21:51,621 | Send setup complete.


In [182]:
model_path = 'dreambooth/output'
my_prompt = f'A highly detailed photograph of sks {class_name} scuba diving in the great barrier reef'
my_prompt = f'A highly detailed photograph of sks {class_name} as Luke Skywalker in a lightsaber battle against Darth Vader, detailed symmetric face, trending'
my_prompt = f'A highly detailed photograph of sks {class_name} as Captain America on an Army recruitment poster, detailed symmetric face'
my_prompt = f'sks {class_name} in a turtleneck on the cover of Vogue magazine, detailed symmetric face'
images = generate_gpu(my_prompt,
                      model_id=model_path,
                      num_images=4, guidance_scale=10,
                      steps=100)

INFO | 2023-01-19 05:16:29,815 | Running sd_generate via SSH
INFO | 2023-01-19 05:16:39,393 | Time to send message: 9.57 seconds


In [ ]:
[display(image) for image in images]